In [1]:
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4,
                       threads_per_worker=4,
                       memory_target_fraction=0.95,
                       memory_limit='8GB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40889,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:45341,Total threads: 4
Dashboard: http://127.0.0.1:35163/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:42283,


In [2]:
import dask.dataframe as dd
import pandas as pd
import re

In [3]:
#df = dd.read_csv("archive-cc-2023-nc.csv.gz", compression='gzip', usecols=['archive_id'])
df = dd.read_csv("archive-cc-2023-nc.csv.gz", blocksize=None, compression='gzip')
df

,archive_id,text_nc
npartitions=1,,
,object,object
,...,...


In [4]:
%%time
df.head()

CPU times: user 1.28 s, sys: 77.5 ms, total: 1.36 s
Wall time: 12.4 s


,archive_id,text_nc
0,MSNBCW_20221029_070000_Dateline,>> she was a young actress starring in the rol...
1,CNNW_20230418_090000_Early_Start_With_Christin...,issued a warrant for his arrest in missouri. c...
2,CNNW_20230504_230000_Erin_Burnett_OutFront,but a source told cnn he will amend his filing...
3,MSNBCW_20221228_210000_Deadline_White_House,really was. from this latest discovery we have...
4,MSNBCW_20230402_210000_PoliticsNation,mar-a-lago tuesday night after his court appea...


In [5]:
%%time
df[df.archive_id.isin(['MSNBCW_20230402_210000_PoliticsNation', 'MSNBCW_20221029_070000_Dateline'])].compute()

CPU times: user 1.16 s, sys: 63.9 ms, total: 1.23 s
Wall time: 11.3 s


,archive_id,text_nc
0,MSNBCW_20221029_070000_Dateline,>> she was a young actress starring in the rol...
4,MSNBCW_20230402_210000_PoliticsNation,mar-a-lago tuesday night after his court appea...


In [6]:
# Define a function for string replacement
def replace_strings(column):
    try:
        return column.apply(lambda x: x if pd.isna(x) else re.sub("b'(.*)'", '\\1', x))
    except Exception as e:
        return column

In [15]:
%%time
first = True
OUTPUT_FILE = '../shared/archive-cc-2023-all-nc.csv.gz'
for i, adf in enumerate(pd.read_csv('../shared/archive-cc-2023.csv.gz', chunksize=1000)):
    print(i)
    adf = adf.apply(lambda col: replace_strings(col), axis=0)
    idents = adf.identifier.tolist()
    cdf = df[df.archive_id.isin(idents)].compute()
    #print(len(cdf))
    adf = pd.merge(adf, cdf, how='left', left_on='identifier', right_on='archive_id')
    del adf['archive_id']
    if first:
        adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
        first = False
    else:
        adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    #if i >= 1:
    #    break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
CPU times: user 15min 32s, sys: 50.7 s, total: 16min 23s
Wall time: 42min 48s
